In [2]:
import pandas as pd
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn import naive_bayes
from sklearn.model_selection import cross_val_score
import librosa

from librosa.feature import melspectrogram
from librosa.effects import trim
import numpy as np

In [20]:
def prepare_song(y, sr):
  y = trim(y)[0]

  if len(y) < 2 * sr + 3:
    y = np.append(y, np.zeros(2 * sr + 3 - len(y)))
  start = 0
  end = len(y)  
  y = y[end//2 - sr: end // 2 + sr]
  return librosa.feature.melspectrogram(y=y, sr=sr, n_mels = 100)

  

(100, 40)


In [27]:
!git add 'pipeline.ipynb'

fatal: pathspec ''pipeline.ipynb'' did not match any files


In [22]:
all_tracks = [None] * 13936

sex = [None] * 13936
i = 0
import csv
with open('C:\\Users\\Xiaomi\\Desktop\\ML-biometry-task-1\\train\\targets.tsv') as file:
    tsv_file = csv.reader(file, delimiter="\t")
    for song_name in tsv_file:
        y, sr = librosa.load('C:\\Users\\Xiaomi\\Desktop\\ML-biometry-task-1\\train\\' + song_name[0] + '.wav', sr = 22050)
        song_pieces = prepare_song(y, sr)
        all_tracks[i] = song_pieces
        sex[i] = int(song_name[1])
        i += 1
        if i % 100 == 0:
            print(i)


100
200
300
400
500
600
700
800
900
1000


KeyboardInterrupt: 

In [ ]:
all_tracks = np.array(all_tracks, dtype='float')
all_tracks.shape

In [ ]:
all_tracks = all_tracks.reshape(-1, 100, 87, 1)
# sex = list(map(int, sex))

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# print(all_tracks[5].shape)
# all_tracks = np.concatenate(all_tracks, axis=0)
X_train, X_test, y_train, y_test = train_test_split(all_tracks, 
                                                    np.array(sex),
                                                    test_size=0.33,
                                                    random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_test, 
                                                y_test,
                                                test_size=0.5,
                                                random_state=42)

In [ ]:
X_train.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 87, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))

model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop, Adam
model.compile(loss='binary_crossentropy',
            #   optimizer=RMSprop(learning_rate=0.001),
                optimizer=Adam(),
              metrics='accuracy')

history = model.fit(X_train, y_train, epochs=20, 
                    validation_data=(X_val, y_val))

In [ ]:
import os
files_and_directories = sorted(os.listdir("C:\\Users\\Xiaomi\\Desktop\\ML-biometry-task-1\\test"))
print(files_and_directories)


In [ ]:
y_test = [None] * len(files_and_directories)
i = 0

for song_name in files_and_directories:
    y, sr = librosa.load('C:\\Users\\Xiaomi\\Desktop\\ML-biometry-task-1\\test\\' + song_name , sr = 22050)
    if len(y) > 2 * sr + 2:
        song_pieces = prepare_song(y, sr)
        y_test[i] =  song_pieces
    else:
        y = np.append(y,np.zeros(2 * sr + 3 - len(y)))
        y_test[i] = prepare_song(y, sr)
    if i % 100 == 0:
        print(i)
    i += 1


In [ ]:
y_test = np.array(y_test)
y_test = y_test.reshape(-1, 100, 87, 1)
y_predict = model.predict(y_test)

In [ ]:
y_predict[-2]


In [ ]:
i = 0
with open('C:\\Users\\Xiaomi\\Desktop\\ML-biometry-task-1\\answer.tsv', 'w') as f:
    for song_name in files_and_directories:
        if y_predict[i][0] > 0:
            f.write(song_name[:-4] + '\t' + '1\n')
        elif y_predict[i][0] <= 0:
            f.write(song_name[:-4] + '\t' + '0\n')
        i += 1